# Class II

Last time:

- Purity
- I/O

The remaining three classes:

- Build + test a psuedo-random number generator (PRNG)
- Build + test a [stream](https://en.wikipedia.org/wiki/Stream_%28computing%29) data structure
- Integrate with Elixir's [StreamData](https://hex.pm/packages/stream_data)

Jupyter:

- Lots of good
- Exceptions are printed as a structure (different from IEx) & stack-trace missing


---

Have a look at this page [Linear Congruential Generator](https://en.wikipedia.org/wiki/Linear_congruential_generator) (especially the picture)

In [1]:
m = 9; a = 2; c = 0
seed = 1

1

In [2]:
seed = Integer.mod(a * seed + c, m)

2

In [3]:
seed = Integer.mod(a * seed + c, m)

4

## Version I

In [4]:
defmodule Generator.LCG do
  def f(seed) do
    m = 9; a = 2; c = 0
    seed = Integer.mod(a * seed + c, m)
  end
end

{:module, Generator.LCG, <<70, 79, 82, 49, 0, 0, 4, 192, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 152, 0, 0, 0, 17, 20, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 76, 67, 71, 8, 95, ...>>, {:f, 1}}

In [5]:
Generator.LCG.f(Generator.LCG.f(Generator.LCG.f(1)))

8

## Aside: Function Literals

In [6]:
9 # A integer "literal"

9

In [7]:
"I'm a string!" # A string "literal"

"I'm a string!"

In [8]:
## An object literal in other languages might look like this
%{name: "Joseph Yiasemides",
  occupation: "Software Engineer"}

%{name: "Joseph Yiasemides", occupation: "Software Engineer"}

In [9]:
fn -> nil end # A function "literal"

#Function<20.99386804/0 in :erl_eval.expr/5>

In [10]:
fu = fn ->
  "I'm inside a function!"
end

#Function<20.99386804/0 in :erl_eval.expr/5>

In [11]:
fu.()

"I'm inside a function!"

In [12]:
identity = fn (x) ->
  x
end

#Function<6.99386804/1 in :erl_eval.expr/5>

In [13]:
compare = fn x, y ->
  cond do
    x < y ->
      -1
    x == y ->
      0
    x > y ->
      +1
  end
end

#Function<12.99386804/2 in :erl_eval.expr/5>

In [14]:
apply(compare, [1,2])

-1

In [15]:
compare.(1,2) ## This is equivalent to the cell above

-1

In [16]:
x = "I'm outside!"
f = fn -> ## This is a "closure"
  x
end
f.()

"I'm outside!"

In [17]:
g = fn ->
  z ## This isn't defined (inside or outside)
end

CompileError: 1

---

## Version II

In [17]:
defmodule Generator.LCG do
  def f(m, a, c) do
    fn (seed) ->
      Integer.mod(a * seed + c, m)
    end
 end
end

{:module, Generator.LCG, <<70, 79, 82, 49, 0, 0, 5, 16, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 164, 0, 0, 0, 18, 20, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 76, 67, 71, 8, 95, ...>>, {:f, 3}}

In [18]:
m = 9; a = 2; c = 0

0

In [19]:
g = Generator.LCG.f(m, a, c)

#Function<0.64772075/1 in Generator.LCG.f/3>

In [20]:
g.(g.(g.(1)))

8

In [21]:
1
|> g.()


2

How about some documentation?

## Version III

In [22]:
modulus = 9; multiplier = 2; increment = 0;
h = Generator.LCG.f(modulus, multiplier, increment)

#Function<0.64772075/1 in Generator.LCG.f/3>

In [23]:
h.(1)

2

In [24]:
defmodule Generator.LCG do
  defstruct [
     :modulus,
     :multiplier,
     :increment
  ]

  def f(p) do
    m = p.modulus
    a = p.multiplier
    c = p.increment
    fn (seed) ->
      Integer.mod(a * seed + c, m)
    end
 end
end

{:module, Generator.LCG, <<70, 79, 82, 49, 0, 0, 8, 112, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 13, 0, 0, 0, 29, 20, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 76, 67, 71, 8, 95, ...>>, {:f, 1}}

In [25]:
%Generator.LCG{}

%Generator.LCG{increment: nil, modulus: nil, multiplier: nil}

In [26]:
p = %Generator.LCG{increment: 0, modulus: 9, multiplier: 2}
h = Generator.LCG.f(p)
h.(h.(h.(1)))

8

In [27]:
defmodule Generator.LCG do
  defstruct [
     modulus:    9,
     multiplier: 2,
     increment:  0
  ]

  def f(p) do
    m = p.modulus
    a = p.multiplier
    c = p.increment
    fn (seed) ->
      Integer.mod(a * seed + c, m)
    end
 end
end

{:module, Generator.LCG, <<70, 79, 82, 49, 0, 0, 8, 116, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 13, 0, 0, 0, 29, 20, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 76, 67, 71, 8, 95, ...>>, {:f, 1}}

In [28]:
%Generator.LCG{}

%Generator.LCG{increment: 0, modulus: 9, multiplier: 2}

In [29]:
p = %Generator.LCG{}
h = Generator.LCG.f(p)
h.(h.(h.(1)))

8

## Version IV

In [30]:
defmodule Generator.LCG do
  defmodule Parameters do
    defstruct [
       modulus:    9,
       multiplier: 2,
       increment:  0
    ]

    def okay?(p) do
      m = p.modulus
      a = p.multiplier
      c = p.increment
      
      x = 0 < m
      y = (0 < a and a < m)
      z = (0 <= c and c < m)

      x and y and z
    end
  end

  def f(p) do
    m = p.modulus
    a = p.multiplier
    c = p.increment
    fn (seed) ->
      Integer.mod(a * seed + c, m)
    end
 end
end

{:module, Generator.LCG, <<70, 79, 82, 49, 0, 0, 6, 96, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 206, 0, 0, 0, 23, 20, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 76, 67, 71, 8, 95, ...>>, {:f, 1}}

In [31]:
p = %Generator.LCG.Parameters{increment: 0, modulus: 9, multiplier: 2}
h = Generator.LCG.f(p)
h.(h.(h.(1)))

8

## Aside: Module Nesting & Name-spacing

In [32]:
defmodule M do
  defmodule N do
    def f do
      "I'm in M.N!"
    end
  end
  def f do
    N.f()
  end
end

{:module, M, <<70, 79, 82, 49, 0, 0, 4, 0, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 128, 0, 0, 0, 14, 8, 69, 108, 105, 120, 105, 114, 46, 77, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, 99, ...>>, {:f, 0}}

In [33]:
M.f()

"I'm in M.N!"

---

## Version V

In [34]:
Generator.LCG.Parameters.okay?(p)

true

In [35]:
defmodule Generator.LCG do
  defmodule Parameters do
    defstruct [
       modulus:    9,
       multiplier: 2,
       increment:  0
    ]

    def okay?(p) do
      m = p.modulus
      a = p.multiplier
      c = p.increment
      
      x = 0 < m
      y = (0 < a and a < m)
      z = (0 <= c and c < m)

      x and y and z
    end
  end

  def instance(p) do
    m = p.modulus
    a = p.multiplier
    c = p.increment
    fn (seed) when seed >= 0 and seed < m ->
      Integer.mod(a * seed + c, m)
    end
 end
end

{:module, Generator.LCG, <<70, 79, 82, 49, 0, 0, 6, 224, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 236, 0, 0, 0, 24, 20, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 76, 67, 71, 8, 95, ...>>, {:instance, 1}}

In [36]:
x = %Generator.LCG.Parameters{
  modulus:    9,
  multiplier: 2,
  increment:  0
}
true = Generator.LCG.Parameters.okay?(x)
f = Generator.LCG.instance(x)

#Function<0.18101733/1 in Generator.LCG.instance/1>

In [37]:
f.(1)

2

In [38]:
f.(-1)

FunctionClauseError: 1

## Aside: Guard Clauses

In [38]:
defmodule G do
  def compare(x, y) when x < y do
    -1
  end
  def compare(x, y) when x === y do
    0
  end
  def compare(x, y) when x > y do
    +1
  end

  def _compare(x, y) do
    cond do
      x < y ->
        -1
      x == y ->
        0
      x > y ->
        +1
    end
  end
end

{:module, G, <<70, 79, 82, 49, 0, 0, 5, 96, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 150, 0, 0, 0, 16, 8, 69, 108, 105, 120, 105, 114, 46, 71, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, 99, ...>>, {:_compare, 2}}

In [39]:
a = G.compare(1, 2) === -1; b = G._compare(1, 2) === -1
a and b

true

---

In [40]:
try do
  f.(16)
rescue
  FunctionClauseError ->
    "Oops!"
end

"Oops!"

In [41]:
defmodule M do
  def f(_) do
    nil
  end
end

{:module, M, <<70, 79, 82, 49, 0, 0, 3, 240, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 121, 0, 0, 0, 14, 8, 69, 108, 105, 120, 105, 114, 46, 77, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, 99, ...>>, {:f, 1}}

## Summary

- **[LS]** The shell, `IEx`, is a good place to wrap our head around what we're doing
- **[FP]** Elixir, like many modern languages, provides a literal for building functions.
  This is the `fn [parameter...] -> [expression...] end` form.
  We've seen the capture operator `&` which has a similar effect
- **[FP]** If a variable defined outside a function literal is used inside it then the function literal pulls in the variable's value.
  This is called closure
- **[FP]** In our case we build generators (just a function) that keep a hold of their parameters (the multiplier, increment, and modulus) and take one argument.
  You can think of the function `Generator.LCG.instance/1` as a factory
- **[SE]** We then introduce structures to make explicit and crystal clear exactly what the parameters are.
  This doesn't rely on the caller to give meaningful variable names
- **[SE]** We went further to wrap the structure in a module of its own.
  This way we exploit Elixir's name-space to spell out exactly what it is that we pass into our "factory" `instance/1`.
  This isn't special to functional programming: writing declaritive code is about naming.
- **[SE]** In version V, the presence of a `Parameters` module meant that we knew where to focus our effort when writing `okay?`.
  Imagine the effect this can have on a much larger codebase.
  An engineer can come along an know exactly where to write or modify a piece of code
- **[LS]** It's common to use function clauses and guards in Elixir.

Prefixes:

- **LS**: language specific
- **FP**: functional programming
- **SE**: software engineer